In [1]:
import MySQLdb
import torch
import numpy as np
import matplotlib.pyplot as plt
import stock
import re
from IPython.display import display, Markdown
import xueqiu
import time

In [2]:
#1. 忽略掉不想要的类型
#2. 忽略掉在一年内有连续跌停的
#3. 忽略掉ST
#符合条件的filter返回True
keywords = [r".*证券.*",r".*银行.*",r".*芯片.*",r".*半导体.*",r".*集成电路.*",r".*互联网.*",
            r".*制药.*",r".*固废.*",r".*通信.*",r".*光电.*",r".*酒.*"]
def descFilter(c):
    for pat in keywords:
        if re.match(pat,c[3]) is not None:
            return True
    return False
def IgnoreCompany(filter=descFilter):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""update company set `ignore`=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
        
    db.query("""select * from descript where name not like '%ST%' and name not like '%B%'""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    for c in companys:
        if filter(c):
            try:
                cursor.execute("""update company set `ignore`=0 where code='%s'"""%(c[2]))
                db.commit()
            except Exception as e:
                print(e)
                db.rollback()
    try:
        cursor.execute("""select count(*) from company where `ignore`=0""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    result = cursor.fetchone()
    db.close()
    print(result)
    
def loadCompany():
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    db.query("""select * from company where `ignore`=0""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    db.close()
    return companys

def loadKline(code,n=120):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    db.query("""select * from company where code='%s'"""%code)
    r = db.store_result()
    company = r.fetch_row()
    db.query("""select volume,open,high,low,close,macd from kd_xueqiu where id=%s order by date desc limit %d"""%(company[0][0],n))
    r = db.store_result()
    k = r.fetch_row(r.num_rows())
    db.query("""select date from kd_xueqiu where id=%s order by date desc limit %d"""%(company[0][0],n))
    r = db.store_result()
    kdate = r.fetch_row(r.num_rows())
    db.close()
    kline = np.array(k).reshape(-1,6)
    return company,np.flip(kline),np.flip(kdate)

# 使用boll 极限宽带小于0.1来选择股票

In [12]:
def bollWidth(k):
    bo20 = stock.bollLineK(k,20)
    lw = (bo20[:,2]-bo20[:,0])/bo20[:,1]
    lwmax = 0
    lwmin = 10
    #boll极限宽小于0.1
    for i in range(len(lw)-10,len(lw)-1):
        if lw[i] > lwmax:
            lwmax = lw[i]
        if lw[i] < lwmin:
            lwmin = lw[i]
    #返回最小的极限宽带和最大的极限宽带
    return lwmin,lwmax,lw[-1],k[-1,4]-k[-2,4],k[-1,4]-bo20[-1,1]

def storeSelectCompany(blist):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""delete from company_select2 where source=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    for c in blist:
        try:
            cursor.execute("""insert ignore into company_select2 values ('%s','%s',1)"""%(c[0],c[1]))
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()        
    db.close()
    
def ouputSelectCompany(str,blist,i=-1):
    display(Markdown("""## %s %d"""%(str,len(blist))))
    for b in blist:
        if i==-1 or b[2]==i:
            display(Markdown("""[%s %s](https://xueqiu.com/S/%s)"""%(b[0],b[1],b[0])))
def showSelectCompany(blist):
    ouputSelectCompany('打开向上',blist,0)
    ouputSelectCompany('打开向下',blist,1)
    ouputSelectCompany('通道上',blist,2)
    ouputSelectCompany('通道下',blist,3)
    
def selectList():
    companys = loadCompany()
    result = []
    for com in companys:
        c,kline,kdate = loadKline(com[1])
        lwmin,lwmax,lwlast,delta,cm20 = bollWidth(kline)
        if lwmax < 0.1 and lwmax/lwmin < 1.5:
            if lwlast > lwmax and lwlast/lwmax>1.1:
                if delta>0:
                    result.append([com[1],com[2],0])
                else:
                    result.append([com[1],com[2],1])
            else:
                if cm20 > 0:
                    result.append([com[1],com[2],2])
                else:
                    result.append([com[1],com[2],3])
    return result

In [13]:
#对当前列表中的进行即使跟踪
#返回当前成交量和昨天的比较
def watch15(code):
    b,json = xueqiu.xueqiuK15(code)
    today_price = 0
    yesterday_price = 0
    today_volume = 0
    yesterday_volume = 0
    if b :
        days = set()
        items = json['data']['item']
        c = time.localtime()
        mins = []
        for i in range(-1,-len(items),-1):
            ts = time.localtime(items[i][0]/1000.)
            if c.tm_mday == ts.tm_mday:
                days.add(ts.tm_mday)
                today_volume += items[i][1]
                if today_price==0:
                    today_price = items[i][5]
                mins.append(ts.tm_hour*60+ts.tm_min)
            elif ts.tm_hour*60+ts.tm_min in mins:
                days.add(ts.tm_mday)
                yesterday_volume += items[i][1]
                if yesterday_price==0:
                    yesterday_price = items[i][5]
    else:
        print(json)
    return today_price,yesterday_price,today_volume,yesterday_volume

In [14]:
bol = selectList()

# 对在BOLL通道内的进行及时检测

In [15]:
def watch15Boll(ls):
    bls = []
    for c in ls:
        tp,yp,tv,yv = watch15(c[0])
        if tp!=0 and yp!=0 and tv!=0 and yv!=0:
            if tv/yv>1.6:
                bls.append([c[0],"%s %.2f %.2f"%(c[1],tv/yv,tp/yp),tv/yv])
    bls = sorted(bls,key=lambda c:c[2],reverse = True)
    ouputSelectCompany('成交量放大',bls)
    return bls

In [20]:
bls = watch15Boll(bol)

## 成交量放大 26

[SZ300334 津膜科技 4.09 0.96](https://xueqiu.com/S/SZ300334)

[SH600175 美都能源 4.01 1.00](https://xueqiu.com/S/SH600175)

[SZ300161 华中数控 3.72 0.98](https://xueqiu.com/S/SZ300161)

[SH601398 工商银行 2.85 0.99](https://xueqiu.com/S/SH601398)

...

[SZ300171 东富龙 1.80 0.97](https://xueqiu.com/S/SZ300171)

[SH601200 上海环境 1.77 0.99](https://xueqiu.com/S/SH601200)

[SH601607 上海医药 1.73 0.99](https://xueqiu.com/S/SH601607)

[SZ000756 新华制药 1.73 1.00](https://xueqiu.com/S/SZ000756)

[SZ300599 雄塑科技 1.62 0.99](https://xueqiu.com/S/SZ300599)

In [19]:
showSelectCompany(bol)

## 打开向上 76

[SH603222 济民制药](https://xueqiu.com/S/SH603222)

[SZ002230 科大讯飞](https://xueqiu.com/S/SZ002230)

## 打开向下 76

## 通道上 76

...

[SH600575 淮河能源](https://xueqiu.com/S/SH600575)

[SZ000516 国际医学](https://xueqiu.com/S/SZ000516)

[SZ002349 精华制药](https://xueqiu.com/S/SZ002349)

[SZ002424 贵州百灵](https://xueqiu.com/S/SZ002424)

[SH600388 龙净环保](https://xueqiu.com/S/SH600388)